### compile
```compile(source, filename, mode[, flags[, dont_inherit]])```
- source    : 字符串或者AST（Abstract Syntax Trees）对象。
- filename  : 代码文件名称，如果不是从文件读取代码则传递一些可辨认的值。
- mode      : 指定编译代码的种类。可以指定为 **exec, eval, single**
- flags     : 变量作用域，局部命名空间，如果被提供，可以是任何映射对象。。
- flags和dont_inherit是用来控制编译源码时的标志

> 参数mode是用来指明那种表示的源码类型；如果是exec类型，表示这是一个序列语句，可以进行运行；如果是eval类型，表示这是一个单一的表达式语句，可以用来计算相应的值出来；如果是single类型，表示这是一个单一语句，采用交互模式执行，在这种情况下，如果是一个表达式，一般会输出结果，而不是打印为None输出。
   

In [1]:
txt = "for i in range(0,6): print(i)" 
c = compile(txt,'','exec')   # 编译为字节代码对象 
print c
exec(c)

<code object <module> at 0000000003FE9830, file "", line 1>
0
1
2
3
4
5


In [2]:
eval(c)

0
1
2
3
4
5


In [3]:
txt = "3 * 4 + 5"
e = compile(txt,'','eval')
eval(e)

17

In [4]:
exec(e)

指定flags为1024时编译为抽象语法树AST

In [5]:
PyCF_ONLY_AST = 1024
txt = "for i in range(0,6): print(i)" 
str_node = compile(txt,'','exec', PyCF_ONLY_AST)
str_node

### eval
```eval(expression, globals=None, locals=None)```
- expression 只能是单个表达式，不支持复杂的代码逻辑，例如赋值操作、循环语句等等
- globals 用于指定运行时的全局命名空间，类型是字典，缺省时使用的是当前模块的内置命名空间。locals 指定运行时的局部命名空间，类型是字典，缺省时使用 globals 的值。两者都缺省时，则遵循 eval 函数执行时的作用域。值得注意的是，这两者不代表真正的命名空间，只在运算时起作用，运算后则销毁。

In [6]:
x = 10
def func():
    y = 20
    a = eval('x + y') # 没有指定命名空间，使用真实作用域，级用x=10
    print('a: ', a) # 这里结果是30
    b = eval('x + y', {'x': 1, 'y': 2}) # 指定了全局作用域，locals为None，默认为globals，所以y=2
    print('x: ' + str(x) + ' y: ' + str(y))
    print('b: ', b) # 这里结果为3
    c = eval('x + y', {'x': 1, 'y': 2}, {'y': 3, 'z': 4})
    print('x: ' + str(x) + ' y: ' + str(y))
    print('c: ', c)

func()

('a: ', 30)
x: 10 y: 20
('b: ', 3)
x: 10 y: 20
('c: ', 4)


可见：
- 当指定了命名空间的时候，变量会在对应命名空间中查找。而且，它们的值不会覆盖实际命名空间中的值
- 当指定了命名空间space，但是所需的变量没有在space中定义，那么会报错，如下：

In [15]:
g = {"x":6}
l = {}
x = 10
def func():
    y = 20
    result = eval("x + y ", g, l)
    print("result:", result)
func()

NameError: name 'y' is not defined

### exec
```exec(object[, globals[, locals]])```
- exec() 的第一个参数不是表达式，而是代码块，这意味着两点：
    - 一是它不能做表达式求值并返回出去
    - 二是它可以执行复杂的代码逻辑，相对而言功能更加强大
    - 例如，当代码块中赋值了新的变量时，该变量可能 在函数外的命名空间中存活下来。
    - 所执行的语句中，如果包含 return 或 yield ，它们产生的值也无法在 exec 函数的外部起作用。
- globals, locals的作用同eval，起到的是白名单的作用，通过限定命名空间的范围，防止作用域内的数据被滥用

In [20]:
x = 1
exec("x = 2 + 2")
print(x)

4


In [27]:
g = {}
l = {}
x = 10
def func():
    y = 20
    exec("x,y=x+1,y+1")
    print x,y
    exec("x,y=x+1,y+1", g, l) # 这个地方一旦赋值了，那么就以g/l为准了
    print x,y
func()

11 21


NameError: name 'x' is not defined

【参考资料】

1. [深度辨析 Python 的 eval() 与 exec()](https://juejin.im/post/5c97885b6fb9a070c11f929e)
2. [eval、exec、globals、locals用法](https://www.cnblogs.com/yyds/p/6276746.html)
3. [在Python中eval,exec和compile有什么区别？](https://codeday.me/bug/20170310/4841.html)